In [19]:
import rasterio
import dask.array as da
from dask_rasterio import read_raster, write_raster
import os
import fnmatch
import numpy as np
import geopandas as gpd
import shapely
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy.spatial as es
import seaborn as sns
import matplotlib.pyplot as plt
import earthpy.plot as ep
import imageio
import rasterstats

In [13]:
input_dir = r'Z:\LWI_StageIV\test'
outputDir= r'Z:\LWI_StageIV\!Accumulated'
cropped_dir = os.path.join(input_dir, 'cropped')
img_dir = os.path.join(cropped_dir, 'img')
outputFilename = input_dir.split('\\')[-1]+'-Inc-Cropped.gif'
sns.set_style("white")
sns.set(font_scale=1.5)

isExist_img = os.path.exists(img_dir)
if not isExist_img:
    os.makedirs(img_dir)

In [14]:
merge_dir=[]
for filename in fnmatch.filter(os.listdir(cropped_dir),'*.01h'): 
    merge_dir.append((os.path.join(cropped_dir, filename)))

merge_dir.sort()

In [28]:
arrayList = []
rasterStats = {}
for raster in merge_dir:
        # print(f'adding {raster} to dask array list: map2array')
        src = rasterio.open(raster)
        array = src.read(1)
        # array[array==0] = np.nan    
        array[array==9999] = np.nan
        arrayList.append(array)
        
        # Get Rasterstats
        stats = rasterstats.zonal_stats(r"Z:\GIS\StageIv Boundary.shp", raster, stats=['min', 'max','mean', 'count', 'nodata'])
        percentMissing = stats[0]['nodata']/stats[0]['count']
        percentMissing = round(percentMissing * 100, 2)
        percentMissing
        stats[0]['percentMissing'] = percentMissing
        rasterStats[raster] = stats
        src.close()
rasterStats

{'Z:\\LWI_StageIV\\test\\cropped\\ST4.2005082000.01h': [{'min': 0.0,
   'max': 19.75,
   'mean': 0.028063334514947417,
   'count': 16926,
   'nodata': 21.0,
   'percentMissing': 0.12}],
 'Z:\\LWI_StageIV\\test\\cropped\\ST4.2005082001.01h': [{'min': 0.0,
   'max': 5.0,
   'mean': 0.00427804331150056,
   'count': 16947,
   'nodata': 0.0,
   'percentMissing': 0.0}],
 'Z:\\LWI_StageIV\\test\\cropped\\ST4.2005082002.01h': [{'min': 0.0,
   'max': 4.0,
   'mean': 0.0005900749395173187,
   'count': 16947,
   'nodata': 0.0,
   'percentMissing': 0.0}]}

In [33]:
# Get the biggest max for the plot colorBar maxValue to be.
bigMax = 0
for raster in rasterStats:
    max = rasterStats[raster][0]['max']
    if (max > bigMax):
        bigMax = max

In [44]:
for i, array in enumerate(arrayList):
    # print (f'writing projected Raster file {projected_raster}')
    # fig, ax = plt.subplots(figsize = (20, 10))
    # im = ax.imshow(array.squeeze(), cmap='rainbow')
    # ep.colorbar(im)
    title = merge_dir[i].split('\\')[-1]
    # ax.set(title=f"{title} Hourly Precip (mm)")
    # ax.set_axis_off()
    img_filename = img_dir + f'\\{title}.png'
    # plt.savefig((img_filename))
    # plt.close()

    crop_shp = gpd.read_file(r"Z:\GIS\StageIv Boundary.shp")
    fig, ax = plt.subplots(figsize=(20, 15))
    la = gpd.read_file("Z:\GIS\Louisiana.shp")
    bbox = la.total_bounds
    la_extent=bbox[[0,2,1,3]]

    bbox_lwi = crop_shp.total_bounds
    lwi_extent = bbox_lwi[[0,2,1,3]]

    # fn = r"Z:\LWI_StageIV\Hurricane Katrina 2005\projected\ST4.2005082913.01h"
    # title = fn.split('\\')[-1]
    # raster = rasterio.open(fn)

    # raster_window = raster.window(*bbox_lwi)
    # array = raster.read(1, window=raster_window)
    array[array==9999] = np.nan
    im = plt.imshow(array, extent=lwi_extent, cmap='rainbow', vmax=bigMax)
    cb = plt.colorbar(im, shrink=.6)
    ax.set(title=f"{title} Incremental Precip (mm)")
    ax.set_axis_off()
    la.boundary.plot(ax=plt.gca(), color='darkgrey')
    plt.savefig((img_filename))
    plt.close()


C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:30: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:30: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
C:\Tools\Anaconda3\envs\py37\lib\site-packages\ipykernel_launcher.py:30: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.


In [ ]:
print (f'Writing plot images to {img_dir}')
for i, array in enumerate(arrayList):
    # print (f'writing projected Raster file {projected_raster}')
    fig, ax = plt.subplots(figsize = (20, 10))
    im = ax.imshow(array.squeeze(), cmap='rainbow')
    ep.colorbar(im)
    title = merge_dir[i].split('\\')[-1]
    ax.set(title=f"{title} Hourly Precip (mm)")
    ax.set_axis_off()
    img_filename = img_dir + f'\\{title}.png'
    plt.savefig((img_filename))
    plt.close()

In [ ]:
crop_shp = gpd.read_file(r"Z:\GIS\StageIv Boundary.shp")
fig, ax = plt.subplots(figsize=(15, 15))
la = gpd.read_file("Z:\GIS\Louisiana.shp")
bbox = la.total_bounds
la_extent=bbox[[0,2,1,3]]

bbox_lwi = crop_shp.total_bounds
lwi_extent = bbox_lwi[[0,2,1,3]]
fn = r"Z:\LWI_StageIV\Hurricane Katrina 2005\projected\ST4.2005082913.01h"
title = fn.split('\\')[-1]
raster = rasterio.open(fn)

raster_window = raster.window(*bbox_lwi)
array = raster.read(1, window=raster_window)
array[array==raster.nodata] = np.nan
im = plt.imshow(array, extent=lwi_extent, cmap='rainbow')
cb = plt.colorbar(im, shrink=.6)
ax.set(title=f"{title} Cumulative Precip (mm)")
ax.set_axis_off()
la.boundary.plot(ax=plt.gca(), color='darkgrey')